# Workflow Manager

This Notebook provides a means of managing preparing and/or running iprPy calculations.  

Note that this Notebook mostly outlines the code/steps associated with with work and is *not* the most optimal means of preparing and running. This is especially true for runners as ideally each runner should be a truly separate process to observe and manage how each is behaving.

For preparing and running on a cluster, the suggestions are:
- Copy the prepare cells below that you wish to use into a Python script.  Submit a job to the cluster for the prepare script.  It only needs to be a serial process, but can take a long time depending on how many calculations are being prepared.
- Submit separate jobs for each runner you wish to be active.  These can easily be based on the iprPy runner command line.

Example prepare Python scripts that correspond to the content below can be found in the bin/prepare/ directory of the iprPy repository.

In [1]:
# import libraries
import numpy as np
import atomman as am

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.11.3


---

## 1. Load the database

The database to use where the calculation records will be added to and to search for existing calculations to skip.

In [2]:
database = iprPy.load_database('master')
print(database)

database style mongo at localhost:27017.iprPy


In [9]:
#local = iprPy.load_database('potentials_local')
#local.copy_records(database, 'potential_LAMMPS', includetar=True, overwrite=False)

488 records to try to copy


copying records: 100%|###############################################################| 488/488 [00:07<00:00, 63.97it/s]

1 records added/updated
1 tars added/updated


---

## 2. Define global prepare terms

All prepare terms are collected into a dictionary making it easy to pass along to the underlying prepare methods.

In [3]:
prepare_terms = {}

### 2.1. Executable terms

These are basic terms that specify executables and some options that are required by most calculations in the workflow

- __lammps_command__ is the primary LAMMPS executable to use.
- __mpi_command__ is the MPI command to use.  Leave {np_per_runner} as a variable.

In [4]:
prepare_terms['lammps_command'] =        'E:/LAMMPS/2020-03-03/bin/lmp_mpi'
prepare_terms['mpi_command'] =           'mpiexec -localonly {np_per_runner}'

### 2.2. Old LAMMPS executables (optional)

Some older implementations of potentials will no longer work with the most current version of LAMMPS.  These options allow for alternate LAMMPS executables to be automatically selected as needed.  Note that this is only important if you want to compare the different versions of a given potential as all current active potentials in the NIST database are compatible with the newest LAMMPS.

- __lammps_command_snap_1__: SNAP version 1 needs LAMMPS between 8 Oct 2014 and 30 May 2017.
- __lammps_command_snap_2__: SNAP version 2 needs LAMMPS between 3 Dec 2018 and 12 June 2019.
- __lammps_command_old__: Some older implementations of potentials need LAMMPS before 30 Oct 2019.

In [5]:
prepare_terms['lammps_command_snap_1'] = 'E:/LAMMPS/2017-01-27/bin/lmp_mpi'
prepare_terms['lammps_command_snap_2'] = 'E:/LAMMPS/2019-06-05/bin/lmp_mpi'
prepare_terms['lammps_command_old'] =    'E:/LAMMPS/2019-06-05/bin/lmp_mpi'

---

## 3. Specify the LAMMPS potentials to use

Any terms in prepare_terms that start with "potential_" are identified as terms that modify which LAMMPS potentials are to be used to prepare the calculations.  All "potential_" terms are passed to database.potdb.get_lammps_potentials() as kwargs with the "potential_" part of their name removed.  Then, only calculations that correspond to the matching LAMMPS potentials returned are prepared.

The most useful related terms are

- __potential_status__ *(None, str or list, optional*) Limits the search by the status of the LAMMPS implementations: "active", "superseded" and/or "retracted".  Most users should set this to 'active
- __potential_id__ *(str or list*) The unique record id(s) labeling the records to parse by.
- __potential_potid__ *(str or list, optional*) The unique record id(s) labeling the associated potential records to parse by.
- __potential_pair_style__ *(str or list, optional*) LAMMPS pair_style(s) to parse by.
- __potential_symbols__ *(str or list, optional*) Model symbol(s) to parse by.  Typically correspond to elements for atomic potential models.
- __potential_elements__ *(str or list, optional*) Element(s) in the model to parse by.

In [6]:
prepare_terms['potential_status'] =      'active'
prepare_terms['potential_id'] = [
    #'2019--Yang-C--Nb--LAMMPS--ipr1',
    '2009--Purja-Pun-G-P--Ni-Al--LAMMPS--ipr1',
    #'2022--Mahata-A--Al-Ni--LAMMPS--ipr1',
]

In [6]:
prepare_terms['potential_status'] =      'active'
#prepare_terms['potential_id'] = [
#    '2022--Alvi-S-M-A-A--Ag--LAMMPS--ipr1',
#]

prepare_terms['potential_pair_style'] = [
    'adp',
    #'aenet',
    'agni',
    'bop',
    'comb3',
    'eam',
    'eam/alloy',
    'eam/cd',
    'eam/fs',
    'eam/he',
    'edip',
    'edip/multi',
    'eim',
    'extep',
    'hybrid',
    'hybrid/overlay',
    #'kim',
    'lcbop',
    'meam',
    'meam/spline',
    #'pinn',
    'polymorphic',
    'reax/c',
    'sw',
    'table',
    'tersoff',
    'tersoff/mod',
    'tersoff/mod/c',
    'tersoff/zbl',
    'vashishta',
]


In [7]:
refs_df = database.get_records_df('reference_crystal')
ref_families = refs_df[refs_df.natoms > 20].id.tolist()
len(ref_families)

1906

---

## 4. Prepare pools

The prepared calculations are divided into separate "pools" of similar calculation styles.  This is done for a variety of reasons.

- Different pools may benefit from a larger number of processors to work with.
- Separating the calculation styles helps in showing the overall progress on obtaining results for the different styles.  
- Some calculation results serve as inputs for later calculations.  Dividing calculations based on where they are positioned in the workflow makes it easier to determine the optimum times to prepare further calculations.

Each master prepare option used below defines a set of default prepare terms and values to use.  These can be overridden by simply adding new terms to prepare_terms that give replacement values.  Additionally, any other calculation/prepare options can be modified using prepare_terms.

Not all possible calculations will be prepared for a given style until all corresponding parent calculations are finished meaning that you should either wait to prepare those calculations or prepare them multiple times. 

The prepare_terms options required for each pool are

- __styles__ lists the iprPy calculation styles to prepare in the pool.  By default, these will use the pre-defined "main" branch, but alternate branches can be selected by giving the branch name after a :.
- __run_directory__ is the name of the specific run directory where the pool is located.  All prepared calculations will be created in this run directory.
- __np_per_runner__ is the number of processors each runner will be assigned to use for the underlying simulations.
- __num_pots__ is the maximum number of potentials to prepare at a time.  Smaller numbers mean that any associated prepares are called more times, but it reduces the number of calculation variations to build and test prior to any calculations being created.

It is possible to prepare multiple pools at the same time by giving lists of values for the four terms above.  However, this Notebook breaks the pools into separate cells for better interactivity and control.

### 4.1. Pool #1: Basic potential evaluations and scans

These are basic potential evaluation methods and initial energy scans.  

Parent calculations:

- None

#### 4.1.1. isolated_atom

Evaluates the energy of a single atom in isolation.

    buildcombos                     lammpspotential potential_file intpot

#### 4.1.2. diatom_scan

Evaluates the energy of a pair of atoms at various interatomic spacings.

    buildcombos                     diatom potential_file intpot
    minimum_r                       0.02 angstrom
    maximum_r                       10.0 angstrom
    number_of_steps_r               500

#### 4.1.3. E_vs_r_scan:bop

Is a variation of E_vs_r_scan specifically for bop potentials where the minimum r value is increased.  __NOTE:__ This should be listed/called before E_vs_r_scan so that bop potentials are prepared with this and not the standard E_vs_r_scan.

    buildcombos                     crystalprototype load_file prototype
    prototype_potential_pair_style  bop
    sizemults                       10 10 10
    minimum_r                       2.0 angstrom
    maximum_r                       6.0 angstrom
    number_of_steps_r               201

#### 4.1.4. E_vs_r_scan

Evaluates the energy of crystal prototypes subjected to a volumetric scan.

    buildcombos                     crystalprototype load_file prototype
    sizemults                       10 10 10
    minimum_r                       0.5 angstrom
    maximum_r                       6.0 angstrom
    number_of_steps_r               276

In [10]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    'isolated_atom',
    'diatom_scan',
    #'E_vs_r_scan:bop',
    'E_vs_r_scan',
])
prepare_terms['run_directory'] = 'master_1'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '100'

# Run master_prepare
database.master_prepare(**prepare_terms)

1 potential ids found
Preparing calculation isolated_atom branch main
Using potential #s 0 to 0

1013 existing calculation records found
1 matching interatomic potentials found
1 calculation combinations to check
1 new records to prepare


preparing: 100%|#########################################################################| 1/1 [00:05<00:00,  5.21s/it]



Preparing calculation diatom_scan branch main
Using potential #s 0 to 0

2192 existing calculation records found
1 matching interatomic potentials found
1 calculation combinations to check
1 new records to prepare


preparing: 100%|#########################################################################| 1/1 [00:05<00:00,  5.23s/it]



Preparing calculation E_vs_r_scan branch main
Using potential #s 0 to 0

28466 existing calculation records found
19 matching crystal prototypes found
1 matching interatomic potentials found
10 calculation combinations to check
10 new records to prepare


preparing: 100%|#######################################################################| 10/10 [00:54<00:00,  5.49s/it]


### 4.2. Pool #2: Crystal relaxations

These perform crystal structure relaxations based on a guess structure and an interatomic potential.  

Parent calculations:

- E_vs_r_scan
- E_vs_r_scan:bop

#### 4.2.1. relax_box

Relaxes a crystal structure by only altering box dimensions to zero pressure while keeping all atoms in the same box-relative positions.

    buildcombos                     atomicreference load_file reference
    buildcombos                     atomicparent load_file parent
    parent_record                   calculation_E_vs_r_scan
    parent_load_key                 minimum-atomic-system
    parent_status                   finished
    sizemults                       10 10 10
    atomshift                       0.05 0.05 0.05
    strainrange                     1e-6

#### 4.2.2. relax_static

Relaxes a crystal structure using energy/force minimization plus a simultaneous box relax.

    buildcombos                     atomicreference load_file reference
    buildcombos                     atomicparent load_file parent
    parent_record                   calculation_E_vs_r_scan
    parent_load_key                 minimum-atomic-system
    parent_status                   finished
    sizemults                       10 10 10
    atomshift                       0.05 0.05 0.05
    energytolerance                 0.0
    forcetolerance                  1e-10 eV/angstrom
    maxiterations                   10000
    maxevaluations                  100000
    maxatommotion                   0.01 angstrom
    maxcycles                       100
    cycletolerance                  1e-10

#### 4.2.3. relax_dynamic

Relaxes a crystal structure using a nph barrostat plus a Langevin thermostat set at 0 K.  This evolves the system while dampening out forces over time.

    buildcombos                     atomicreference load_file reference
    buildcombos                     atomicparent load_file parent
    parent_record                   calculation_E_vs_r_scan
    parent_load_key                 minimum-atomic-system
    parent_status                   finished
    sizemults                       10 10 10
    atomshift                       0.05 0.05 0.05
    temperature                     0.0
    integrator                      nph+l
    thermosteps                     1000
    runsteps                        10000
    equilsteps                      0

In [11]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    'relax_box',
    'relax_static',
    'relax_dynamic',
])
prepare_terms['run_directory'] = 'master_2'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '100'

# Run master_prepare
database.master_prepare(**prepare_terms)

1 potential ids found
Preparing calculation relax_box branch main
Using potential #s 0 to 0

106652 existing calculation records found
6587 matching atomic references found
1 matching interatomic potentials found
1 matching interatomic potentials found
10 matching atomic parents found
36 calculation combinations to check
36 new records to prepare


preparing: 100%|#######################################################################| 36/36 [03:10<00:00,  5.29s/it]



Preparing calculation relax_static branch main
Using potential #s 0 to 0

204766 existing calculation records found
6587 matching atomic references found
1 matching interatomic potentials found
1 matching interatomic potentials found
10 matching atomic parents found
36 calculation combinations to check
36 new records to prepare


preparing: 100%|#######################################################################| 36/36 [03:18<00:00,  5.52s/it]



Preparing calculation relax_dynamic branch main
Using potential #s 0 to 0

107462 existing calculation records found
6587 matching atomic references found
1 matching interatomic potentials found
1 matching interatomic potentials found
10 matching atomic parents found
36 calculation combinations to check
36 new records to prepare


preparing: 100%|#######################################################################| 36/36 [03:14<00:00,  5.40s/it]


### 4.3. Pool #3: Further crystal relaxations

This performs further crystal relaxations on the results of pool #2. 

Parent calculations

- relax_dynamic

#### 4.3.1. relax_static:from_dynamic

Takes the resulting structures of relax_dynamic and subjects them to an energy/force minimization plus box relaxation.

    buildcombos                     atomicarchive load_file archive
    archive_record                  calculation_relax_dynamic
    archive_branch                  main
    archive_load_key                final-system
    archive_status                  finished
    sizemults                       1 1 1
    energytolerance                 0.0
    forcetolerance                  1e-10 eV/angstrom
    maxiterations                   10000
    maxevaluations                  100000
    maxatommotion                   0.01 angstrom
    maxcycles                       100
    cycletolerance                  1e-10

In [12]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    'relax_static:from_dynamic'
])
prepare_terms['run_directory'] = 'master_3'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '100'

# Run master_prepare
database.master_prepare(**prepare_terms)

1 potential ids found
Preparing calculation relax_static branch from_dynamic
Using potential #s 0 to 0

204802 existing calculation records found
1 matching interatomic potentials found
36 matching atomic archives found
36 calculation combinations to check
36 new records to prepare


preparing: 100%|#######################################################################| 36/36 [06:43<00:00, 11.20s/it]


### 4.4. Pool #4: Crystal space group analysis

These evaluate the crystal space group information for the relaxed structures computed above and for the initial prototype and DFT structures used.

Parent calculations

- relax_box
- relax_static
- relax_static:from_dynamic

#### 4.4.1. crystal_space_group:prototype

Evaluates the crystal space group information for the prototype structures.  Only needs to be done once per prototype.

    buildcombos                     crystalprototype load_file proto

#### 4.4.2. crystal_space_group:reference

Evaluates the crystal space group information for DFT relaxed structures.  Only needs to be done once per structure.

    buildcombos                     atomicreference load_file ref

#### 4.4.3. crystal_space_group:relax

Takes the resulting structures of relax_dynamic and subjects them to an energy/force minimization plus box relaxation.

    buildcombos                     atomicarchive load_file archive1
    buildcombos                     atomicarchive load_file archive2
    archive1_record                 calculation_relax_static
    archive1_load_key               final-system
    archive1_status                 finished
    archive2_record                 calculation_relax_box
    archive2_load_key               final-system
    archive2_status                 finished

In [7]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    #'crystal_space_group:prototype',
    #'crystal_space_group:reference',
    'crystal_space_group:relax',
])
prepare_terms['run_directory'] = 'master_4'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '100'

# Run master_prepare
database.master_prepare(**prepare_terms)

1 potential ids found
Preparing calculation crystal_space_group branch relax
Using potential #s 0 to 0

222522 existing calculation records found
52 matching atomic archives found
32 matching atomic archives found
84 calculation combinations to check
84 new records to prepare


preparing: 100%|#######################################################################| 84/84 [08:15<00:00,  5.90s/it]


### 4.5 SEPARATE SCRIPT Identify relaxed crystals

Once all of the above calculations are done for a given potential, run the code in the "3a. Process relaxed crystal results.ipynb" to generate relaxed_crystal records.  The relaxed_crystal records are then used as the inputs for the subsequent calculation methods.

Notes on relaxed_crystal records:
- Always limit to crystals with a "good" standing.  The standing term only has two options "good" and "bad", with the bad being identified as duplicates of the good.
- The automatic processes typically limit to crystals found using the "dynamic" method. The "box" and "static" crystals are much more likely to become unstable with perturbations resulting in junk results where the wanted structure transforms into something else. In addition to not being useful, these transformed results can be computational sinks for minimization relaxations.
- Even with the above limitations, there is no guarantee that the structure will not transform into something else upon relaxation.

### 4.6. Pool #5: Elastic constants and phonons

These evaluate properties of the identified 0K relaxed crystal structures.

Parent records

- relaxed_crystals

#### 4.6.1. elastic_constants_static

Estimates the static elastic constants of a crystal structure using small displacements.  Three different strains are used to obtain three independent estimates, which can differ for some potentials/crystals.

    buildcombos                     atomicparent load_file parent
    parent_record                   relaxed_crystal
    parent_standing                 good
    sizemults                       10 10 10
    maxiterations                   5000
    maxevaluations                  10000
    strainrange                     1e-6
    strainrange                     1e-7
    strainrange                     1e-8

#### 4.6.2. phonon

Uses phonopy to evaluate phonons for a crystal structure and attempt to perform the quasi-harmonic analysis.

    buildcombos                     atomicparent load_file parent
    parent_record                   relaxed_crystal
    parent_method                   dynamic
    parent_standing                 good
    sizemults                       3 3 3

In [7]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    'elastic_constants_static',
    'phonon'
])
prepare_terms['run_directory'] = 'master_5'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '50'

#prepare_terms['parent_family'] = ref_families

# Run master_prepare
database.master_prepare(**prepare_terms)

1 potential ids found
Preparing calculation elastic_constants_static branch main
Using potential #s 0 to 0

132336 existing calculation records found
1 matching interatomic potentials found
17 matching atomic parents found
51 calculation combinations to check
48 new records to prepare


preparing: 100%|#######################################################################| 48/48 [04:24<00:00,  5.52s/it]



Preparing calculation phonon branch main
Using potential #s 0 to 0

15413 existing calculation records found
1 matching interatomic potentials found
2 matching atomic parents found
2 calculation combinations to check
2 new records to prepare


preparing: 100%|#########################################################################| 2/2 [00:15<00:00,  7.53s/it]


### 4.7. Pool #6: Static defect formation energies

These evaluate the static formation energies of simple crystal defects.

Parent records

- relaxed_crystals

#### 4.7.1. point_defect_static

Evaluates the formation energy and elastic dipole tensor associated with a single point defect or small cluster of point defects.  Also returns evaluation parameters to help identify if the defect has transformed into a different configuration. 

    buildcombos                     atomicparent load_file parent
    parent_record                   relaxed_crystal
    parent_method                   dynamic
    parent_standing                 good
    buildcombos                     defect pointdefect_file
    defect_record                   point_defect
    sizemults                       12 12 12
    forcetolerance                  1e-8

#### 4.7.2. surface_energy_static

Evaluates the formation energy of a cleaved free surface.

    buildcombos                     atomicparent load_file parent
    parent_record                   relaxed_crystal
    parent_method                   dynamic
    parent_standing                 good
    buildcombos                     defect surface_file
    defect_record                   free_surface
    sizemults                       3 3 8

#### 4.7.3. stacking_fault_map_2D

Evaluates the generalized stacking fault energy and relaxation distances associated with a full 2D scan of a fault plane.  

    buildcombos                     atomicparent load_file parent
    parent_record                   relaxed_crystal
    parent_method                   dynamic
    parent_standing                 good
    buildcombos                     defect stackingfault_file
    defect_record                   stacking_fault
    sizemults                       5 5 10
    stackingfault_num_a1            30
    stackingfault_num_a2            30

In [ ]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    'point_defect_static',
])
prepare_terms['run_directory'] = 'master_6'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '50'

# Run master_prepare
database.master_prepare(**prepare_terms)

### 4.8. Pool #7: Dislocation core structures

These construct and relax atomic configurations that contain a single dislocation.  This provides not only a means of evaluating the relaxed core structure, but serves as the starting point for dislocation slip and interaction simulations.

NOTE: for these styles, different branches are used for different defect types so that appropriate settings (typically sizemults) can be specified for each unique defect.

Parent records

- elastic_constants_static

#### 4.8.1. dislocation_monopole

Creates a dislocation monopole configuration.

    buildcombos                     atomicparent load_file parent
    parent_record                   calculation_elastic_constants_static
    parent_load_key                 system-info
    parent_strainrange              1e-7
    buildcombos                     defect dislocation_file
    defect_record                   dislocation
    dislocation_boundarywidth       3
    dislocation_boundaryscale       True
    dislocation_boundaryshape       cylinder
    annealtemperature               10
    annealsteps                     10000000
    maxiterations                   10000
    maxevaluations                  100000
    
##### 4.8.1.1. dislocation_monopole:fcc_edge_100
    
    parent_family                   A1--Cu--fcc
    defect_id                       A1--Cu--fcc--a-2-110--90-edge--{100}
    sizemults                       1 80 116

##### 4.8.1.2. dislocation_monopole:bcc_screw

    parent_family                   A2--W--bcc
    defect_id                       A2--W--bcc--a-2-111--0-screw--{110}
    sizemults                       1 48 80

##### 4.8.1.3. dislocation_monopole:bcc_edge

    parent_family                   A2--W--bcc
    defect_id                       A2--W--bcc--a-2-111--90-edge--{110}
    sizemults                       1 68 80

##### 4.8.1.4. dislocation_monopole:bcc_edge_112

    parent_family                   A2--W--bcc
    defect_id                       A2--W--bcc--a-2-111--90-edge--{112}
    sizemults                       1 68 48
    
#### 4.8.2. dislocation_periodic_array

Creates a periodic array of dislocations configuration.

    buildcombos                     atomicparent load_file parent
    parent_record                   calculation_elastic_constants_static
    parent_load_key                 system-info
    parent_strainrange              1e-7
    buildcombos                     defect dislocation_file
    defect_record                   dislocation
    dislocation_boundarywidth       3
    dislocation_boundaryscale       True
    dislocation_duplicatecutoff     1 angstrom
    annealtemperature               10
    annealsteps                     10000000
    maxiterations                   10000
    maxevaluations                  100000
    
##### 4.8.2.1. dislocation_periodic_array:fcc_edge_mix
    
    parent_family                   A1--Cu--fcc
    defect_id                       A1--Cu--fcc--a-2-110--90-edge--{111}
    sizemults                       1 200 50
    dislocation_onlylinear          False

##### 4.8.2.2. dislocation_periodic_array:fcc_screw

    parent_family                   A1--Cu--fcc
    defect_id                       A1--Cu--fcc--a-2-110--0-screw--{111}
    sizemults                       1 200 50
    dislocation_onlylinear          True

### 4.9. Pool #8: Dynamic relaxations

These run MD relaxations on the "relaxed" crystals at a variety of temperatures

Parent records

- relaxed_crystals

#### 4.6.1. relax_dynamic:at_temp

Runs an npt relaxation MD simulation at a specific temperature between 100 and 3000 K

    buildcombos                     atomicparent load_file parent
    parent_record                   relaxed_crystal
    parent_method                   dynamic
    parent_standing                 good
    sizemults                       10 10 10
    atomshift                       0.05 0.05 0.05
    integrator                      npt
    thermosteps                     100
    runsteps                        1000000
    restartsteps                    50000
    equilsteps                      0
    temperature                     100
    temperature                     200
    ...
    temperature                     2900
    temperature                     3000

In [7]:
# Specify master prepare options
prepare_terms['styles'] = ' '.join([
    'relax_dynamic:at_temp',
    #'phonon'
])
#prepare_terms['parent_family'] = ['A1--Cu--fcc', 'L1_2--AuCu3']
prepare_terms['run_directory'] = 'master_7'
prepare_terms['np_per_runner'] = '1'
prepare_terms['num_pots']      = '50'

# Run master_prepare
database.master_prepare(**prepare_terms)

1 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 0

107678 existing calculation records found
1 matching interatomic potentials found
38 matching atomic parents found
1140 calculation combinations to check
1020 new records to prepare


preparing: 100%|###################################################################| 1020/1020 [52:11<00:00,  3.07s/it]


### 4.X.  Further styles coming soon...

## 5. Runner

Once calculations have been prepared, you can then start runner jobs to perform them.  

Options for managing runners:

- Use the cell below to call runner() for the database.  This will perform one calculation at a time until finished or stopped.  Not recommended unless you only want one runner active at any given time.
- Open a separate terminal for each runner you wish to be active and call the "iprPy runner" command with the specific database and run directory you want each to use.  This isolates each runner in action and allows for runners to operate on the same or different databases and run directories.
- Submit runner jobs to a cluster that has access to the run directory and the database.


In [8]:
database.runner(run_directory='master_5')

Runner started with pid 76392
5aecc5fa-1adf-495f-abe7-280a3d63badd
sim calculated successfully

a46b7675-4de1-4329-921b-4d3d76ccd3aa
sim calculated successfully

5455cace-628e-48cf-b47a-eaea49f49d00
sim calculated successfully

430824cf-767f-4fe4-9dfe-8ae54a27821a
sim calculated successfully

03342a9c-91d1-41c7-9810-89676a1d27ed
sim calculated successfully

19cc775b-a12b-4a94-aebf-5d7839e465ef
sim calculated successfully

e787c46a-7c2a-47f2-be35-af77669fbb1e
sim calculated successfully

8056e334-58f1-44fd-bbfd-f944e0f397ac
sim calculated successfully

47ab5417-c4f9-4f02-955b-76eb4a5ec2e9
sim calculated successfully

d351386e-f6c0-4ced-91c3-63a7ea48b7d6
sim calculated successfully

8c8da058-53ed-47af-b513-175eb7d8cfbc
error: Did not assign all restart atoms correctly (../read_restart.cpp:533)
Last command: read_restart initial.restart

09a56813-4ccb-4297-bcf3-0935bc28952b
sim calculated successfully

ba39199d-f50e-4e0d-bcf4-e28e7a743cb2
sim calculated successfully

150c16d6-8578-42ec-86